# LightGBM

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import KFold

In [ ]:
params = {
    'num_leaves': 12,
    # 'min_child_samples': 500,
    'objective': 'binary', #2値分類: 'binary', 多値分類: 'multiple', 回帰: 'regression'
    'metric': 'None', # Noneにしないとbinary_loglossでEarlyStoppingがかかる
    'max_depth': -1,
    'learning_rate': 0.01,
    "boosting_type": "gbdt", #dart
    "subsample": 0.7,
    "verbosity": -1,
    'early_stopping_rounds': 300,
    'n_estimators': 10000
}

In [ ]:
def lightgbms(X, X_test, y, params, NFOLDS=5, random_state=2020):
    
    #==== KFOLD ====
    #KFold
    #folds = KFold(n_splits=NFOLDS, shuffle=True, random_state=random_state)
    #splits = folds.split(X, y)
    
    #StratiedKFold
    #folds = StratifiedKFold(n_splits=NFOLDS, shuffle=True, random_state=random_state)
    #splits = folds.split(X, y)
    
    #GroupKFold
    #folds = GroupKFold(n_splits=NFOLDS)
    #splits = folds.split(X, y, train_df["chip_id"])
    
    preds = pd.DataFrame()
    train_preds = pd.DataFrame()

    columns = X.columns

    y_preds = np.zeros(X_test.shape[0])
    y_oof = np.zeros(X.shape[0])

    score = 0
    scores = []

    feature_importances = pd.DataFrame()
    feature_importances['feature'] = columns

    for fold_n, (train_index, val_index) in enumerate(splits):
        
        # split train, Valid
        X_train, X_val = X[columns].iloc[train_index], X[columns].iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]
        
        #==== Sampling Space ====
        
        
        
        #==== lightbgm dataset ====
        dtrain = lgb.Dataset(X_train, y_train)
        dval = lgb.Dataset(X_val, y_val)
        
        #==== train ====
        clf = lgb.train(params,
                        dtrain,
                        valid_sets=[dtrain, dval],
                        #categorical : categorical_feature=col_category,
                        # custam metrics : feval=pr_auc,
                        verbose_eval=100,
                       )
        
        # feature importances
        feature_importances[f'fold_{fold_n + 1}'] = clf.feature_importance()

        # stack
        y_pred_val = clf.predict(X_val)
        y_oof[val_index] = y_pred_val
        
        # score
        score = average_precision_score(y_val, y_pred_val)
        scores.append(score)
        

        y_preds += clf.predict(X_test) / NFOLDS

        train_preds[f"Fold {fold_n + 1}"] = clf.predict(X)

        preds[f"Fold {fold_n + 1}"] = clf.predict(X_test)

        del X_train, X_val, y_train, y_val
        #gc.collect()
        
    print(np.mean(scores))
    
    return { "y_oof": y_oof, 
            "y_pred" : y_preds, 
            "feature_importances" : feature_importances, 
            "score" : scores,
           }

In [ ]:
results = lightgbms(X, test, y, params)

# ラベル化

In [ ]:
y_pred = []
 
for x in preds:
    y_pred.append(np.argmax(x))

# feature_importance

In [ ]:
feature_importances = pd.read_csv(path+'feature_importance/NO13_2_features.csv')
feature_importances['average'] = feature_importances[[f'fold_{fold_n + 1}' for fold_n in range(10)]].mean(axis=1)

plt.figure(figsize=(16, 16))
sns.barplot(data=feature_importances.sort_values(by='average', ascending=False).head(50), x='average', y='feature');
plt.title('250 TOP feature importance over {} folds average'.format(10))
#plt.savefig('TOP250_feature_importance.png')
plt.show()

# 予測値の分布

In [ ]:
import seaborn as sns
sns.distplot(results["y_pred"])